In [1]:
import aoc

data = aoc.read("day20.txt")

In [2]:
# % = flipflop. starts as OFF. in: high, NOTHING.in: low: flips. off -> on: out high pulse. on -> off: out low pulse
# & = conjunction; remember most recent pulse per input module. on input: update memory for input. All high -> output low. else: output high
# broadcaster: multiply in to all destination modules
# button: send low to broadcaster
# pulses handled in the order they are sent

In [3]:
flipflops = set()
conjunctions = set()
outputs = set()
connected = set()
connections = {}
state = {}
for line in data.splitlines():
    sender, receivers = line.split("->")

    if sender.startswith("%"):
        name_sender = sender[1:].strip()
        state[name_sender] = False
        flipflops.add(name_sender)
    elif sender.startswith("&"):
        name_sender = sender[1:].strip()
        conjunctions.add(name_sender)
    elif sender.strip() == "broadcaster":
        name_sender = "broadcaster"
    else:
        assert False
    connections[name_sender] = []
    for receiver in receivers.split(","):
        name_receiver = receiver.strip()
        connected.add((name_sender, name_receiver))
        connections[name_sender].append(name_receiver)

state = {"broadcaster": {"type": "broadcaster"}}
for flpflp in flipflops:
    state[flpflp] = {"type": "flipflop", "on": False}
for cnjnctn in conjunctions:
    state[cnjnctn] = {"type": "conjunction", "received_high_input": {}}

for connection in connected:
    sending_mod, receiving_mod = connection
    if (
        receiving_mod not in flipflops
        and receiving_mod not in conjunctions
        and receiving_mod != "broadcaster"
    ):
        state[receiving_mod] = {"type": "output"}
    if receiving_mod in conjunctions:
        state[receiving_mod]["received_high_input"][sending_mod] = False

In [54]:
import copy

initial_state = copy.deepcopy(state)

# Part 1

In [6]:
from collections import deque

In [ ]:
n_lo = 0
n_hi = 0

state = copy.deepcopy(initial_state)
for n_pressed in range(1_000):
    q = deque([("lo", "broadcaster", "button")])
    while q:
        type_pulse, receiver, sender = q.popleft()
        if type_pulse == "lo":
            n_lo += 1
        elif type_pulse == "hi":
            n_hi += 1

        module_state = state[receiver]
        type_module = module_state["type"]

        if type_module == "output":
            continue

        elif type_module == "broadcaster":
            outgoing_pulse = "lo"

        elif type_module == "flipflop":
            if type_pulse == "hi":
                continue
            was_on = state[receiver]["on"]
            outgoing_pulse = "lo" if was_on else "hi"
            state[receiver]["on"] = not was_on

        elif type_module == "conjunction":
            state[receiver]["received_high_input"][sender] = (
                True if type_pulse == "hi" else False
            )
            if all(v for v in state[receiver]["received_high_input"].values()):
                outgoing_pulse = "lo"
            else:
                outgoing_pulse = "hi"
        for new_receiver in connections[receiver]:
            q.append((outgoing_pulse, new_receiver, receiver))
print(n_lo * n_hi)

# Part 2

In [ ]:
# Just looking for the signal will take forever. Let's investigate what cycle is needed
def find_sending_machines(module):
    sending_machines = []
    for cn in connected:
        if cn[1] == module:
            if cn[0] in conjunctions:
                sending_machines.append(f"&{cn[0]}")
            elif cn[0] in flipflops:
                sending_machines.append(f"#{cn[0]}")
            else:
                assert False
    return sending_machines


sending = {}
now = ["_rx"]  # Slice off the first part because it's the module type in other modules
for lvls_deep in range(3):
    new_sending = []
    for mod in now:
        sending[mod] = find_sending_machines(mod[1:])
        print(f"{mod}: {sending[mod]}")
        new_sending.extend(sending[mod])
    now = new_sending.copy()
print(now)

In [ ]:
import tqdm

state = copy.deepcopy(initial_state)

# If the following send lo pulse, then `rx` will too
sent_lo_pulses = {"lg": set(), "st": set(), "gr": set(), "bn": set()}
for n_pressed in tqdm.tqdm(range(100_000)):
    q = deque([("lo", "broadcaster", "button")])
    while q:
        type_pulse, receiver, sender = q.popleft()
        if sender in sent_lo_pulses.keys() and type_pulse == "lo":
            sent_lo_pulses[sender].add(n_pressed)
        if receiver == "rx":
            if type_pulse == "lo":
                print(n_pressed)
                break
        if type_pulse == "lo":
            n_lo += 1
        elif type_pulse == "hi":
            n_hi += 1

        module_state = state[receiver]
        type_module = module_state["type"]

        if type_module == "output":
            continue

        elif type_module == "broadcaster":
            outgoing_pulse = "lo"

        elif type_module == "flipflop":
            if type_pulse == "hi":
                continue
            was_on = state[receiver]["on"]
            outgoing_pulse = "lo" if was_on else "hi"
            state[receiver]["on"] = not was_on

        elif type_module == "conjunction":
            state[receiver]["received_high_input"][sender] = (
                True if type_pulse == "hi" else False
            )
            if all(v for v in state[receiver]["received_high_input"].values()):
                outgoing_pulse = "lo"
            else:
                outgoing_pulse = "hi"
        for new_receiver in connections[receiver]:
            q.append((outgoing_pulse, new_receiver, receiver))

In [ ]:
cycles = {}
for key, n_pressed in sent_lo_pulses.items():
    start_cycle = min(n_pressed)
    length_cycle = (max(n_pressed) - min(n_pressed)) / (len(n_pressed) - 1)
    cycles[key] = (start_cycle, length_cycle)
print(cycles)

import math

if all(v[0] == v[1] - 1 for v in cycles.values()):  # python starts counting at 0
    print(math.lcm(*[int(v[1]) for v in cycles.values()]))